# AutoGenU for Jupyter

An Automatic C++ Code Generator for Nonlinear Model Predictive Control (NMPC)  

---  

## Formulation of the optimal control problem: Pendubot

### Model description
<img src="https://raw.github.com/wiki/mayataka/CGMRES/images/fig_pendubot.png" width=25%>   
The above figure shows the model of a pendubot. The equation of the motion can be derived by the Lagrange's method as
$$ \ddot{\theta}_1 = \ddot{\theta}_1 (\theta_1, \theta_2, \dot{\theta}_1, \dot{\theta}_2) ,$$
$$ \ddot{\theta}_2 = \ddot{\theta}_2 (\theta_1, \theta_2, \dot{\theta}_1, \dot{\theta}_2) .$$
We ommit to describe its details here because it is too complicated.
We assume that the input of the pendubot is limited as  
$$u_{\rm min} \leq u \leq u_{\rm max} , $$
where $u_{\rm min}, u_{\rm max} \in \mathbb{R}$.

### Formulation of the optimal control problem
We define the state vector as
$$ x = \begin{bmatrix} \theta_1 \\ \theta_2 \\ \dot{\theta}_1 \\ \dot{\theta}_2 \end{bmatrix} .$$
The state equation is then given as 
$$\dot{x} = f(t, x, u) = \begin{bmatrix} \dot{\theta}_1 \\ \dot{\theta}_2 \\ \ddot{\theta}_1 (\theta_1, \theta_2, \dot{\theta}_1, \dot{\theta}_2) \\ \ddot{\theta}_2 (\theta_1, \theta_2, \dot{\theta}_1, \dot{\theta}_2) \end{bmatrix} .$$
Next, we set the cost function 
$$ J = \varphi(t, x) + \int_{t}^{t+T} L(t, x, u) .$$
The objective is to invert the pendubot and stabilize it. In other words, we want to converge the state of the pendubot to
$$x_{\rm ref} := \begin{bmatrix} \pi \\ 0 \\ 0 \\ 0 \end{bmatrix}.$$  
To achieve this goal, we set the terminal cost as
$$\varphi(t,x) = \frac{1}{2} (x - x_{\rm ref})^{\rm T} Q_{\rm terminal} (x - x_{\rm ref}) ,$$  
where $Q_{\rm terminal} = {\rm diag}\left\{ {q_{\rm terminal}}_1, {q_{\rm terminal}}_2, {q_{\rm terminal}}_3, {q_{\rm terminal}}_4 \right\}$ and ${q_{\rm terminal}}_1, {q_{\rm terminal}}_2, {q_{\rm terminal}}_3, {q_{\rm terminal}}_4$ are positive real constants. We also set the stage cost as  
$$L(t, x, u) = \frac{1}{2} (x - x_{\rm ref})^{\rm T} Q (x - x_{\rm ref}) + \frac{1}{2} r u^2,$$  
where $Q = {\rm diag} \left\{ q_1, q_2, q_3, q_4 \right\}$ with positive real constants $q_1, q_2, q_3, q_4$, and $r$ is positive real constant.

---

# Generate C++ code for optimal control problem (OCP)

## Import modules

In [ ]:
import autogenu

## Set dimensions and define `AutoGenU`
- `nx`: Dimension of the state vector $x$   
- `nu`: Dimension of the control input vector $u$  
- `ocp_name`: Name of the optimal control problem (OCP). Used as the name of the directory containing the generated C++ source files.

In [ ]:
nx = 4
nu = 1
ocp_name = 'pendubot'
ag = autogenu.AutoGenU(ocp_name, nx, nu)

## Generate t, x, and u, necessary variables to formulate of the optimal control problem
- `t`: Time parameter $t$  
- `x`: The state vector $x$  
- `u`: The control input vector $u$  

In [ ]:
t = ag.define_t()
x = ag.define_x()
u = ag.define_u()

## Define variables used in the state equation, constraints, and cost function
- Define a scalar variable whose name is "var" as  
`var = ag.define_scalar_var('var') `

- Define scalar variables whose names are "var\_1", "var\_2", ..., "var\_n" as  
`var_1, var_2, ..., var_n = ag.define_scalar_vars('var_1', 'var_2', ..., 'var_n')`

- Define array variables whose name is "vec" and dimension is dim_vec as  
`vec = ag.define_array_var('vec', dim_vec)`

In [ ]:
# Variables used in the state function
m1, m2, l1, l2, d1, d2, J1, J2, g = ag.define_scalar_vars('m1', 'm2', 'l1', 'l2', 'd1', 'd2', 'J1', 'J2', 'g')

# Variables used in the constraints on the control input
# u_min, u_max, dummy_weight = ag.define_scalar_vars('u_min', 'u_max', 'dummy_weight')

# Variables used in the cost function
q = ag.define_array_var('q', nx)
q_terminal = ag.define_array_var('q_terminal', nx)
x_ref = ag.define_array_var('x_ref', nx)
r = ag.define_array_var('r', nu)

## Define the state equation, constraints, the stage cost, and the terminal cost
- `f`: The state equation $ f(t, x, u)$  
- `C`: The equality constraisnts $C(t, x, u) = 0 $  
- `h`: The inequality constraisnts $h(t, x, u) \leq 0$ considered by semi-smooth Fischer-Burumeister method  
- `L`: The stage cost $L(t, x, u)$  
- `phi`: The terminal cost $\phi (t, x)$  

Note: array indices start with 0

In [ ]:
# Import symbolic math from sympy. If you need more functions, import same as below.
from sympy import sin, cos, tan, exp, log, sinh, cosh, tanh, diff, sqrt

In [ ]:
# Define the state equation
f = [x[2], 
     x[3],
     -(sin(x[0] + x[1]) * d2 * g * m2 + g * (d1 * m1 + l1 * m2) * sin(x[0]) - 0.2e1 * d2 * (x[2] + x[3] / 0.2e1) * l1 * x[3] * m2 * sin(x[1]) - u[0]) / (0.2e1 * d2 * m2 * l1 * cos(x[1]) + d1 * d1 * m1 + d2 * d2 * m2 + l1 * l1 * m2 + J1 + J2), 
     (g * d2 * l1 * m2 * (d1 * m1 + l1 * m2) * sin(x[0] - x[1]) / 0.2e1 - d2 * d2 * g * l1 * m2 * m2 * sin(x[0] + 0.2e1 * x[1]) / 0.2e1 - (d1 * d1 * m1 - d1 * l1 * m1 / 0.2e1 + l1 * l1 * m2 / 0.2e1 + J1) * m2 * g * d2 * sin(x[0] + x[1]) - l1 * l1 * m2 * m2 * d2 * d2 * (pow(x[2], 0.2e1) + x[2] * x[3] + pow(x[3], 0.2e1) / 0.2e1) * sin(0.2e1 * x[1]) - l1 * m2 * d2 * ((pow(x[2] + x[3], 0.2e1) * d2 * d2 + pow(x[2], 0.2e1) * l1 * l1) * m2 + (d1 * d1 * m1 + J1 + J2) * pow(x[2], 0.2e1) + 0.2e1 * J2 * x[2] * x[3] + J2 * pow(x[3], 0.2e1)) * sin(x[1]) + g * (d2 * d2 * l1 * m2 * m2 / 0.2e1 + (d1 * d2 * d2 * m1 + J2 * l1) * m2 + J2 * d1 * m1) * sin(x[0]) - u[0] * (d2 * m2 * l1 * cos(x[1]) + d2 * d2 * m2 + J2)) / (0.2e1 * d2 * m2 * l1 * cos(x[1]) + d1 * d1 * m1 + d2 * d2 * m2 + l1 * l1 * m2 + J1 + J2) / (d2 * d2 * m2 + J2)]

# Define the constraints
# C = [u[0]**2 + u[1]**2 - ((u_max-u_min)**2)/4]
C = []

# Define the inequality constraints considered using the semi-smooth Fischer-Burmeister method
h = []
       
# Define the stage cost
L = sum(q[i]*(x[i] - x_ref[i])**2 for i in range(nx))/2 + (r[0] * u[0]**2)/2  

# Define the terminal cost
phi = sum(q_terminal[i]*(x[i] - x_ref[i])**2 for i in range(nx))/2

In [ ]:
# Set functions
ag.set_functions(f, C, h, L, phi)

## Define the control input bound constraints
The box constraints on the control input $u_{\rm min} \leq u \leq u_{\rm max}$ are treated in a special manner.
Specifically, it is transformed into $$u^2 + u_{\rm dummy}^2 - (\frac{u_{\rm max} - u_{\rm min}}{2})^2 = 0$$.  
Further, $u_{\rm dummy}$ and associated Lagrange multiplier are eliminated in Newton-type iteration efficiently in the multiple-shooting solver.  
Set such box constraints by specifying
- `uindex`: index of the constrained control input element
- `umin`: Minimum value (lower bound) of the control input element
- `umax`: Maximum value (upper bound) of the control input element
- `dummy_weight`: Weight parameter of the dummy (slack) variable

In [ ]:
ag.add_control_input_bounds(uindex=0, umin=-5.0, umax=5.0, dummy_weight=0.1)

## Set Parameters
Set values for all parameters you defined.
- Set `value` for a scalar variable whose name is "var" as  
`ag.set_scalar_var('var', value) `

- Set `value_1`, `value_2`, ..., `value_n`, for scalar variables whose names are "var\_1", "var\_2", ..., "var\_n" as  
`ag.set_scalar_vars(['var_1', value_1], ['var_2', value_2], ..., ['var_n', value_n])`

- Set array variables `var_1`, `var_2`, ..., `var_n` whose name is "vec" and dimension is n as  
`ag.define_array_var('vec', [var_1, var_2, ..., var_n])`

In [ ]:
ag.set_scalar_vars(['m1', 0.2], ['m2', 0.7], ['l1', 0.3], ['l2', 0.3], 
                   ['d1', 0.15], ['d2', 0.257], ['J1', 0.006], ['J2', 0.051], ['g', 9.80665])
                #    ['u_min', -5], ['u_max', 5], ['dummy_weight', 0.1])
ag.set_array_var('q', [1, 1, 0.1, 0.1])
ag.set_array_var('r', [0.1])
ag.set_array_var('q_terminal', [1, 1, 0.1, 0.1])
ag.set_array_var('x_ref', ['M_PI', 0, 0, 0])

## Generate C++ codes of the definition of the optimal control problem
Generate `ocp.hpp` that defines the optimal control problem (OCP).  
- `simplification`: The flag for simplification. If `True`, symbolic functions are simplified. However, if functions are too complicated, it takes too much time. Default is `False`.  
- `common_subexpression_elimination`: The flag for common subexpression elimination. If `True`, common subexpressions in fxu, phix, hx, and hu are eliminated when `ocp.hpp` is generated. Default is `False`. 

In [ ]:
simplification = False
common_subexpression_elimination = True

ag.generate_ocp_definition(simplification, common_subexpression_elimination)

---  
# Generate C++ codes of NMPC solver and numerical simulation  

## Set NLP type  
Set which NLP formulation you use. (hint: `MultipleShooting` works better for typical nonlinear problems.)
- `MultipleShooting` (recommended) : The multiple shooting based C/GMRES method with condensing of the state and the Lagragne multipliers with respect to the state equation.
- `SingleShooting` : The single shooting method, i.e., the original C/GMRES method.

In [ ]:
nlp_type = autogenu.NLPType.MultipleShooting
ag.set_nlp_type(nlp_type)

## Set horizon length 

- `Tf`, `alpha`: Parameters for the length of the horizon. If $\alpha > 0$, time-varying horizon length $T(t) = T_f (1 - e^{-\alpha t})$ is used. Otherwise, $T(t)$ is fixed by $T_f$. 

In [ ]:
Tf = 1.0
alpha = 0.0

ag.set_horizon_params(Tf, alpha)

## Set parameters for the C/GMRES method

- `sampling_time`: Sampling period.
- `N`: The number of the discretization grids of the horzion.  
- `finite_difference_epsilon`: Step length of a finite difference approximations of hessian-vector products in C/GMRES.   
- `zeta`: A stabilization parameter of the C/GMRES method. Basically, must be set by the reciprocal of the sampling time.  
- `kmax`: The maximam number of the iteration of the GMRES.

In [ ]:
sampling_time = 0.001
N = 50
finite_difference_epsilon = 1.0e-08
zeta = 1000  
kmax = 5

ag.set_solver_params(sampling_time, N, finite_difference_epsilon, zeta, kmax)

## Set parameters for the initialization of the solution  
- `solution_initial_guess`: The initial guess of the solution of the optimal control problem (OCP) for initialization of the solution of NMPC.   
- `tolerance`: The residual tolerance of the solution of the OCP for the initialization of the solution of NMPC. The Newton iteration terminates when the optimality error is less than this velue.  
- `max_iterations`: The maxmum number of Newton iteration for the initialization of the solution of NMPC.  

In [ ]:
solution_initial_guess = [0.01]
tolerance  = 1.0e-06
max_iterations = 50

ag.set_initialization_params(solution_initial_guess, tolerance, max_iterations)

## Set parameters for numerical simulation
- `initial_time`: Initial time of the numerical simulation.  
- `initial_state`: Initial state vector of the system.  
- `simulation_length`: Simulation length.  

In [ ]:
initial_time = 0  
initial_state = [0, 0, 0, 0]  
simulation_length = 10

ag.set_simulation_params(initial_time, initial_state, simulation_length) 

## Generate main.cpp and CMakeLists.txt

In [ ]:
ag.generate_main()
ag.generate_cmake()

## Build and run simulation

- `generator`: An optional variable for Windows user to choose the generator of CMake. If 'MSYS', then 'MSYS Makefiles' is used. If 'MinGW', then 'MinGW Makefiles' is used. The default value is 'Auto' and the generator is selected automatically. If sh.exe exists in your PATH, MSYS is choosed, and otherwise MinGW is used. 
- `remove_build_dir`: If `True`, the existing build directory is removed. If `False`, the build directory is not removed. Need to be set `True` if you change the generator. Default is `False`.

In [ ]:
generator = 'Auto'  
remove_build_dir = False
ag.git_submodule_update()
ag.build_main(generator, remove_build_dir)
ag.run_simulation()

## Plot the simulation results

In [ ]:
%matplotlib inline

In [ ]:
plotsim = autogenu.PlotSimulation(ocp_name)
plotsim.set_scales(2,5,2)
# plotsim.show_plots()
plotsim.save_plots()

## Draw animations of the simulation results

In [ ]:
anim = autogenu.TwoLinkArm(ocp_name)
anim.set_skip_frames(10)
# anim.generate_animation()

## Build Python interface

- `generator`: An optional variable for Windows user to choose the generator of CMake. If 'MSYS', then 'MSYS Makefiles' is used. If 'MinGW', then 'MinGW Makefiles' is used. The default value is 'Auto' and the generator is selected automatically. If sh.exe exists in your PATH, MSYS is choosed, and otherwise MinGW is used. 
- The python interfaces will be installed at `~/.local/lib/python3.x`. If you use MacOSX or Windows, add it to the Python PATH as 
```
export PYTHONPATH=$PYTHONPATH:$HOME/.local/lib/python3.x/site-packages
```
where `python3.x` is your python version

In [ ]:
generator = 'Auto'  
ag.generate_python_bindings()
ag.git_submodule_update()
ag.build_python_interface(generator)

## Install Python interface

Install Python interfaces.
The usage of Python interfaces is introduced in `examples/python` directory.

In [ ]:
ag.install_python_interface()